In [8]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RepeatedKFold

In [32]:
# train_df = pd.read_csv('../feature_data/train/featureInput_noChampionRole_ZSCORED.csv')
train_df = pd.read_csv('trial_train_data.csv')
# test_df = pd.read_csv('../feature_data/test/featureInput_noChampionRole_ZSCORED.csv')
test_df = pd.read_csv('trial_test_data.csv')

In [34]:
train_df.head(3)

,Unnamed: 0,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,raPlayerRole,...,rsPlayerChampion,bCoopPlayer,rCoopPlayer,bCoopChampion,rCoopChampion,vsPlayer,vsChampion,bTeamColor,rTeamColor,bResult
0,1,0.289738,-0.257036,1.060552,0.377305,0.278699,-0.133841,0.350029,-0.112189,0.713630,...,0.690359,0.441921,0.096551,0.134805,1.112304,-0.175703,0.510275,0.121245,-0.151772,1.0
1,2,-0.263885,-0.766333,-0.260659,-0.853495,-0.950215,-0.332546,3.260819,-1.208941,-0.316147,...,0.346412,-0.698948,-0.656359,-1.708774,0.131480,-1.581668,0.359287,-0.173614,2.230129,0.0
2,3,-0.358951,0.020762,0.699159,-0.409335,0.500586,0.974318,0.453523,0.311440,0.535215,...,0.870778,-0.171815,-0.720948,0.224274,-0.419115,0.677840,0.532583,1.890399,-1.884064,1.0


In [36]:
# drop_cols = list(train_df.filter(regex='ChampionRole$').columns)
drop_cols = ['Unnamed: 0']

In [38]:
train_df = train_df.drop(drop_cols, axis=1)
train_df.columns

Index(['btPlayerRole', 'bjPlayerRole', 'bmPlayerRole', 'baPlayerRole',
       'bsPlayerRole', 'rtPlayerRole', 'rjPlayerRole', 'rmPlayerRole',
       'raPlayerRole', 'rsPlayerRole', 'btPlayerChampion', 'bjPlayerChampion',
       'bmPlayerChampion', 'baPlayerChampion', 'bsPlayerChampion',
       'rtPlayerChampion', 'rjPlayerChampion', 'rmPlayerChampion',
       'raPlayerChampion', 'rsPlayerChampion', 'bCoopPlayer', 'rCoopPlayer',
       'bCoopChampion', 'rCoopChampion', 'vsPlayer', 'vsChampion',
       'bTeamColor', 'rTeamColor', 'bResult'],
      dtype='object')

In [40]:
len(list(train_df.columns))

29

In [42]:
test_df = test_df.drop(drop_cols, axis=1)
len(list(test_df.columns))

29

In [44]:
y_train = train_df.loc[:, 'bResult']
X_train = train_df.loc[:, train_df.columns != 'bResult']
y_test = test_df.loc[:, 'bResult']
X_test = test_df.loc[:, test_df.columns != 'bResult']

In [17]:
X_train.count

<bound method DataFrame.count of       btPlayerRole  bjPlayerRole  bmPlayerRole  baPlayerRole  bsPlayerRole  \
0         0.252250     -0.113933      1.010727      0.434609      0.245249   
1         0.107836     -0.258477      0.248977     -0.715370     -0.487638   
2        -0.363338      0.955698      0.776230     -0.188098      1.248147   
3         0.178569      0.045067     -0.025138     -0.175134      0.548104   
4         2.831061      0.661353      0.758049      1.205383      0.490318   
...            ...           ...           ...           ...           ...   
6818      0.920830      0.753336     -1.787310     -0.131543     -0.857938   
6819     -0.263513     -1.270290     -1.278238     -3.327397     -0.055620   
6820     -1.650736     -0.308239     -3.314526     -3.327397     -1.760546   
6821     -0.156803     -0.258477      1.466323     -0.953493      0.345539   
6822     -3.358087     -3.293916     -2.805454     -2.816349     -2.061415   

      rtPlayerRole  rjPlayerRo

In [46]:
gnb = GaussianNB()

In [48]:
y_train.dtypes

dtype('float64')

In [50]:
gnb.fit(X_train, y_train)

GaussianNB()

In [52]:
# apparently the model must be fit prior to calling this
priors = gnb.predict_proba(X_test)
priors

array([[2.12630756e-02, 9.78736924e-01],
       [9.24459024e-04, 9.99075541e-01],
       [2.95721037e-03, 9.97042790e-01],
       ...,
       [4.18566493e-02, 9.58143351e-01],
       [7.02206701e-01, 2.97793299e-01],
       [8.89990494e-01, 1.10009506e-01]])

In [54]:
y_predict = gnb.predict(X_test)
mean_accuracy = gnb.score(X_test, y_test)
mean_accuracy

0.6569920844327177

In [56]:
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.6569920844327177


In [29]:
len(priors)

759

In [58]:
# tenfold training
rkf = RepeatedKFold(n_repeats=5, n_splits=10, random_state=3)

In [60]:
import numpy as np

In [62]:
X_array = X_train.to_numpy()
y_array = y_train.to_numpy()

In [64]:
accuracies = []

In [66]:
best_probs = {'prob_blue': 0, 'prob_red': 0}

In [68]:
best_accuracy = 0

In [70]:
for train_index, test_index in rkf.split(X_train):
    # print(f'    Train: index={train_index}')
    # print(f'    Test: index={test_index}')
    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]
    prob_blue = len([x for x in list(y_fold_train) if x == 1]) / len(list(y_fold_train))
    prob_red = len([x for x in list(y_fold_train) if x == 0]) / len(list(y_fold_train))
    print(f'probability that blue wins: {prob_blue}')
    print(f'probability that red wins: {prob_red}')
    model = GaussianNB()
    model.fit(X_fold_train, y_fold_train)
    fold_prediction = model.predict(X_fold_test)
    accuracy = accuracy_score(y_fold_test, fold_prediction)
    accuracies.append(accuracy)
    print(f'accuracy: {accuracy}')
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_probs['prob_blue'] = prob_blue
        best_probs['prob_red'] = prob_red

probability that blue wins: 0.5386744829832275
probability that red wins: 0.4613255170167725
accuracy: 0.828696925329429
probability that blue wins: 0.53965152255333
probability that red wins: 0.4603484774466699
accuracy: 0.8125915080527086
probability that blue wins: 0.5430711610486891
probability that red wins: 0.45692883895131087
accuracy: 0.8257686676427526
probability that blue wins: 0.5420941214785866
probability that red wins: 0.45790587852141346
accuracy: 0.8228404099560761
probability that blue wins: 0.5421686746987951
probability that red wins: 0.4578313253012048
accuracy: 0.843108504398827
probability that blue wins: 0.5437968088570498
probability that red wins: 0.45620319114295016
accuracy: 0.8167155425219942
probability that blue wins: 0.5411917942038424
probability that red wins: 0.4588082057961576
accuracy: 0.8328445747800587
probability that blue wins: 0.5424943015304461
probability that red wins: 0.4575056984695539
accuracy: 0.8152492668621701
probability that blue win

In [72]:
accuracies

[0.828696925329429,
 0.8125915080527086,
 0.8257686676427526,
 0.8228404099560761,
 0.843108504398827,
 0.8167155425219942,
 0.8328445747800587,
 0.8152492668621701,
 0.7976539589442815,
 0.843108504398827,
 0.8272327964860908,
 0.8184480234260615,
 0.8111273792093704,
 0.8155197657393851,
 0.8284457478005866,
 0.8357771260997068,
 0.8181818181818182,
 0.8372434017595308,
 0.8328445747800587,
 0.8108504398826979,
 0.8506588579795022,
 0.8301610541727672,
 0.8228404099560761,
 0.7935578330893118,
 0.8108504398826979,
 0.8181818181818182,
 0.8548387096774194,
 0.8167155425219942,
 0.8167155425219942,
 0.8240469208211144,
 0.8184480234260615,
 0.8038067349926794,
 0.8272327964860908,
 0.8272327964860908,
 0.8181818181818182,
 0.8548387096774194,
 0.8284457478005866,
 0.8211143695014663,
 0.8123167155425219,
 0.8211143695014663,
 0.8140556368960469,
 0.8125915080527086,
 0.7994143484626647,
 0.8594436310395315,
 0.8152492668621701,
 0.8269794721407625,
 0.8284457478005866,
 0.8284457478005

In [74]:
len(accuracies)

50

In [76]:
sum(accuracies) / 50

0.8234187193810298

In [78]:
best_accuracy

0.8594436310395315

In [80]:
best_probs

{'prob_blue': 0.5425826412636379, 'prob_red': 0.45741735873636213}

In [82]:
gnb = GaussianNB(priors=[best_probs['prob_red'], best_probs['prob_blue']])

In [84]:
gnb.fit(X_train, y_train)

GaussianNB(priors=[0.45741735873636213, 0.5425826412636379])

In [86]:
tuned_prediction = gnb.predict(X_test)

In [88]:
accuracy_w_priors = accuracy_score(y_test, tuned_prediction)
accuracy_w_priors

0.6569920844327177